### Question 2 - Top 3 Cities

  For each country, compute the top 3 cities with best air quality and the top 3 cities with poorest air quality, updated weekly, i.e., averaged over a week (7 days).

 **Requirement**: Solve this question using Spark Core, Spark Dataframes and Spark SQL.

In [1]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive') # Faz o mount da drive

Mounted at /content/drive


In [2]:
#@title Install Pyspark
!pip install --quiet pyspark # Faz a instalação do Pyspark

     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 52.6 MB/s 


In [4]:
#@title Dataset
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-01-0*.csv > files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-01-1*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-01-2*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-01-3*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-02-0*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-02-1*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-02-2*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-03-0*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-03-1*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-03-2*.csv >> files
!head -10000 /content/drive/MyDrive/projeto_spbd/sds011-2020-03-3*.csv >> files

In [9]:
#@title Resolution using Spark SQL
import math
import pyspark
import json
import string
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import datetime
from operator import *
from pyspark import pandas

spark = SparkSession.builder.master('local[*]').appName('/content/files').getOrCreate()
sc = spark.sparkContext

try :
    custom_schema = StructType([StructField("sen_id", StringType(), True),StructField("file_id", StringType(), True),StructField("lat", FloatType(), True),StructField("lon", FloatType(), True),StructField("timestamp", TimestampType(), True),StructField("p1", FloatType(), True),StructField("p2", FloatType(), True),])
    # É criado um schema
    main_data = spark.read.schema(custom_schema).load('/content/files', sep=';', header=False, format="csv") # O dataset principal é atribuído ao schema criado
    aux_data=spark.read.json('/content/drive/MyDrive/projeto_spbd/sensors_all.json') # Carregamento do dataset secundário (JSON)
    main_data.createOrReplaceTempView('maindata') # Criada uma Temporary View para o dataset principal
    aux_data.createOrReplaceTempView('auxdata') # Criada uma Temporary View para o dataset secundário
    # Join data
    joint_data=spark.sql('SELECT sen_id, country, city, timestamp, p1, p2 FROM maindata INNER JOIN auxdata ON sensor_id=sen_id') # É feito o INNER JOIN entre os dois datasets, pelo sensor_id
    joint_data.createOrReplaceTempView('full_data') # Criada uma Temporary View para o dataset inteiro, já depois de ser feito o INNER JOINT
    query1=spark.sql('SELECT country, city, WEEKOFYEAR(timestamp) AS week, AVG(p1+p2) AS metric FROM full_data GROUP BY country, city, WEEKOFYEAR(timestamp) ORDER BY week, metric DESC')
    # O query1 faz o SELECT do country, city, semana do ano, média da soma dos valores de p1 e p2.
    # Faz o GROUP BY por country, city e week, à semelhança do que foi feito no Spark DataFrame.
    # Faz o ORDER BY week e metric, de forma descendente.
    query1.createOrReplaceTempView('full_data_2') # Criada uma Temporary View, com os resultados do query1
    # Top_cities
    top_cities=spark.sql('SELECT * FROM (SELECT country, city, week, metric, RANK() OVER (PARTITION BY country, week ORDER BY metric DESC) AS rank FROM full_data_2) AS a WHERE rank <=3')
    # O top_cities faz o SELECT do country, city, semana, metric e rank.
    # O rank é feito pelos valores da metric, tendo em conta a partição país e semana. Ou seja, para cada country e week, são apresentados os valores da metric (de forma descendente), bem como a cidade a que corresponde.
    # São apresentados apenas os valores para os rankings menores ou iguais a 3, ou seja, os 3 maiores valores para cada conjunto country/semana.
    top_cities.createOrReplaceTempView('full_data_3') # Criada uma Temporary View com os resultados do top_cities
    # Tail_cities
    tail_cities=spark.sql('SELECT * FROM (SELECT country, city, week, metric, RANK() OVER (PARTITION BY country, week ORDER BY metric ASC) AS rank FROM full_data_2) AS a WHERE rank <=3')
    # O tail_cities faz o SELECT do country, city, semana, metric e rank.
    # O rank é feito pelos valores da metric, tendo em conta a partição país e semana. Ou seja, para cada country e week, são apresentados os valores da metric (de forma ascendente), bem como a cidade a que corresponde.
    # São apresentados apenas os valores para os rankings menores ou iguais a 3, ou seja, os 3 menores valores para cada conjunto country/semana.
    tail_cities.createOrReplaceTempView('full_data_4') # Criada uma Temporary View com os resultados do tail_cities
    print("---------- TOP 3 CITIES WITH POOREST AIR QUALITY ----------")
    top_cities.show() # Apresenta os valores do top_cities
    print("---------- TOP 3 CITIES WITH BEST AIR QUALITY ----------")
    tail_cities.show() # Apresenta os valores do tail_cities
except Exception as err:
    print(err)
    sc.stop()

---------- TOP 3 CITIES WITH POOREST AIR QUALITY ----------
+-------+-----------------+----+------------------+----+
|country|             city|week|            metric|rank|
+-------+-----------------+----+------------------+----+
| France|           Luitré|   1|2097.0089213053384|   1|
| France|        Gallardon|   1| 886.9899520874023|   2|
| France|          Tullins|   1|340.68750071525574|   3|
| France|           Luitré|   2| 2999.800048828125|   1|
| France|        Fonsorbes|   2|160.16167163848877|   2|
| France|          Crusnes|   2| 144.1007126399449|   3|
| France|           Luitré|   3| 2999.800048828125|   1|
| France|         Péchabou|   3|238.21727440573952|   2|
| France|           Anglet|   3|103.99766685962678|   3|
| France|           Luitré|   4|  759.272006225586|   1|
| France|Serémange-Erzange|   4|276.13205500210034|   2|
| France|            Passy|   4|180.86461580716647|   3|
| France|        Tinténiac|   5| 339.0488944186105|   1|
| France|    Bois-Colombes| 